In [1]:
#! python2
# -*- coding: utf-8 -*-
import pprint
import pymongo
import datetime
import numpy as np
from pymongo import MongoClient

from scipy import spatial

import pylab

import os

import progressbar

db_name = 'twitter'
predict_col_name = 'event_list'
true_col_name = 'after_process_replab'

event_size = [10,15]
author_size = [10,15]

In [2]:
from sklearn import metrics
from sklearn.cluster import DBSCAN
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics.pairwise import cosine_similarity

from igraph import *
import igraph
import math
from operator import itemgetter

In [3]:
def get_sim(doc_a,doc_b):
    return 1 - spatial.distance.cosine(doc_a, doc_b)

In [4]:
def get_midnight(time):
    return time.replace(minute=0, hour=0, second=0, microsecond=0)

In [5]:
if __name__ == '__main__':
    client = MongoClient()
    db = client[db_name]

    result = db[predict_col_name].create_index([('ts', pymongo.ASCENDING)])
    cursor = db[predict_col_name].find({})

    bar = progressbar.ProgressBar(maxval=cursor.count()+1, widgets=[progressbar.Bar('#', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    data=1
    predict_event = {}
    predict_event_id = {}
    alltime = []
    for doc in cursor:
        data += 1
        bar.update(data)
        
        date = doc['time']
        description = doc['description']
        group_id = doc['group_id']
        if group_id > 0:
            true_name = doc['true_name']
            tweet_id =doc['tweet_id']
            size = doc['size']
            past = doc['past']
            a_size = doc['author_size']

#             print size,past
            if not predict_event.has_key(date):
                predict_event[date] = {}
                predict_event_id[date] = {}
            if not predict_event[date].has_key(description):
                predict_event[date][description] = {}
                predict_event_id[date][description] = {}
            for i in event_size:
                key = 'e_' + str(i)
                if not predict_event[date][description].has_key(key):
                    predict_event[date][description][key] = []
                    predict_event_id[date][description][key] = []
                
                if size >= i and len(past) == 0:
                    predict_event[date][description][key].append(true_name)
                    predict_event_id[date][description][key].extend(tweet_id)
            
            for i in author_size:
                key = 'a_' + str(i)
                if not predict_event[date][description].has_key(key):
                    predict_event[date][description][key] = []
                    predict_event_id[date][description][key] = []
                
                if a_size >= i and len(past) == 0:
                    predict_event[date][description][key].append(true_name)
                    predict_event_id[date][description][key].extend(tweet_id)
        
        alltime.append(date)
    
    alltime = set(alltime)
        
    bar.finish()

[########################################################################] 100%


In [6]:
if __name__ == '__main__':
    client = MongoClient()
    db = client[db_name]

    result = db[true_col_name].create_index([('ts', pymongo.ASCENDING)])
    cursor = db[true_col_name].find({'ts':{'$gte':1338508800}}).sort([('ts', pymongo.ASCENDING)])
    
    all_event = {}
    all_event_id = {}
    true_event_id_2 = {}

    bar = progressbar.ProgressBar(maxval=cursor.count()+1, widgets=[progressbar.Bar('#', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    data=1
    for doc in cursor:
        data += 1
        bar.update(data)
        
        ts = doc['ts']
        entity_id = doc['entity_id']
        topic = doc['topic']
        tweet_id = doc['tweet_id']
        topic_dense = doc['topic_dense']
        datetime_object = datetime.datetime.fromtimestamp(ts)
        time_gap = get_midnight(datetime_object)
        
        if not all_event.has_key(time_gap):
            all_event[time_gap] = []
            all_event_id[time_gap] = {}
            true_event_id_2[time_gap] = []
        true_event_id_2[time_gap].append(tweet_id)
#         if tweet_id =='208604422492786688':
#             print 'in',time_gap
        if topic_dense == 1:
#         if True:
            if not (topic in all_event[time_gap]):
                 all_event[time_gap].append(topic)
            if not (all_event_id[time_gap].has_key(topic)):
                all_event_id[time_gap][topic] = []
            all_event_id[time_gap][topic].append(tweet_id)

        
    bar.finish()
#     print len(date)

[########################################################################] 100%


In [7]:
starttime = datetime.datetime(2012, 6, 1)
usetime = []
true_event = {}
true_event_id = {}


for day in sorted(alltime):
    if day >= starttime:
        usetime.append(day)
        true_event[day] = []
        true_event_id[day] = []

for now_time, event_list in all_event.items():
    old_time = now_time - datetime.timedelta(days=1)
    for event in event_list:
        if all_event.has_key(old_time):
            if not (event in all_event[old_time]):
                if  true_event.has_key(now_time):
                    true_event[now_time].append(event)
                    true_event_id[now_time].extend( all_event_id[now_time][event] )
        else:
            if  true_event.has_key(now_time):
                true_event[now_time].append(event)
                true_event_id[now_time].extend( all_event_id[now_time][event] )

In [8]:
# Recall = How many relevantitems are selected?
# Precision = How many selecteditems are relevant?
def cal_recall_precision(true, predict):
    recall, precision = 0,0
    if len(true) == 0 and len(predict) == 0:
        recall = 1
        precision = 1
    elif len(true) == 0:
        recall = 0
        precision = 0
    elif len(predict) == 0:
        recall = 0
        precision = 1
    else:
        r = 0
        p = 0
        for x in true:
            if x in predict:
                r += 1
        recall = 1.0*r/len(true)

        for x in predict:
            if x in true:
                p += 1

        precision = 1.0*p/len(predict)
    
    return recall,precision

In [9]:
all_score = {}
all_score_id = {}
bar = progressbar.ProgressBar(maxval=len(usetime)+1, widgets=[progressbar.Bar('#', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
data=1
for day in usetime:
    data += 1
    bar.update(data)
    for description,d_list in predict_event[day].items():
        if not all_score.has_key(description):
            all_score[description] = {}
            all_score_id[description] = {}

        for key,d_event in d_list.items():
            if not all_score[description].has_key(key):
                all_score[description][key] = {}
                all_score[description][key]['recall'] = []
                all_score[description][key]['precision'] = []
            
                all_score_id[description][key] = {}
                all_score_id[description][key]['recall'] = []
                all_score_id[description][key]['precision'] = []
            recall, precision = cal_recall_precision(true_event[day], d_event)
            all_score[description][key]['recall'].append(recall)
            all_score[description][key]['precision'].append(precision)

            recall, precision = cal_recall_precision(true_event_id[day], predict_event_id[day][description][key])
            all_score_id[description][key]['recall'].append(recall)
            all_score_id[description][key]['precision'].append(precision)
bar.finish()

[########################################################################] 100%


In [10]:
for description,list_data in all_score.items():
    for type_key, score_data in list_data.items():
        for score_key,score in score_data.items():
            print description, type_key,score_key, sum(score)/len(score)
        print ''

Affinity Propagation e_15 recall 0.167791666667
Affinity Propagation e_15 precision 0.348714285714

Affinity Propagation a_10 recall 0.114125
Affinity Propagation a_10 precision 0.198740546218

Affinity Propagation a_15 recall 0.170291666667
Affinity Propagation a_15 precision 0.353214285714

Affinity Propagation e_10 recall 0.116625
Affinity Propagation e_10 precision 0.199053046218

DBSCAN e_15 recall 0.297583333333
DBSCAN e_15 precision 0.74125

DBSCAN a_10 recall 0.256208333333
DBSCAN a_10 precision 0.572797619048

DBSCAN a_15 recall 0.295083333333
DBSCAN a_15 precision 0.755

DBSCAN e_10 recall 0.258708333333
DBSCAN e_10 precision 0.563958333333

Markov Cluster e_15 recall 0.259916666667
Markov Cluster e_15 precision 0.46026984127

Markov Cluster a_10 recall 0.205541666667
Markov Cluster a_10 precision 0.306241452991

Markov Cluster a_15 recall 0.269916666667
Markov Cluster a_15 precision 0.48076984127

Markov Cluster e_10 recall 0.200541666667
Markov Cluster e_10 precision 0.2959

In [11]:
# remove noise topic('')
# >=10 and no past using true_name
# Affinity Propagation recall 0.29092060088
# Affinity Propagation precision 0.258155993447

# DBSCAN recall 0.463127769353
# DBSCAN precision 0.672125483693

# Markov Cluster recall 0.371064243498
# Markov Cluster precision 0.375932078098



# >=15 and no past using true_name
# Affinity Propagation recall 0.332974190276
# Affinity Propagation precision 0.336004368349

# DBSCAN recall 0.643948906262
# DBSCAN precision 0.811596778015

# Markov Cluster recall 0.503245681711
# Markov Cluster precision 0.515159908848

In [12]:
for description,list_data in all_score_id.items():
    for type_key, score_data in list_data.items():
        for score_key,score in score_data.items():
            print description, type_key, score_key, sum(score)/len(score)
    #         print score
        print ''

Affinity Propagation e_15 recall 0.306619781604
Affinity Propagation e_15 precision 0.351491276994

Affinity Propagation a_10 recall 0.306285334723
Affinity Propagation a_10 precision 0.198530102975

Affinity Propagation a_15 recall 0.308807281604
Affinity Propagation a_15 precision 0.356275548385

Affinity Propagation e_10 recall 0.309722834723
Affinity Propagation e_10 precision 0.198694535034

DBSCAN e_15 recall 0.318897773239
DBSCAN e_15 precision 0.741493437937

DBSCAN a_10 recall 0.280116132973
DBSCAN a_10 precision 0.572435617487

DBSCAN a_15 recall 0.315772773239
DBSCAN a_15 precision 0.756028811854

DBSCAN e_10 recall 0.282928632973
DBSCAN e_10 precision 0.562834104673

Markov Cluster e_15 recall 0.389135687905
Markov Cluster e_15 precision 0.459522717424

Markov Cluster a_10 recall 0.372892361768
Markov Cluster a_10 precision 0.300804584025

Markov Cluster a_15 recall 0.396219021238
Markov Cluster a_15 precision 0.479529800786

Markov Cluster e_10 recall 0.368725695101
Markov

In [13]:
# remove noise topic('')
# >=10 and no past using tweet_id
# Affinity Propagation recall 0.287937947412
# Affinity Propagation precision 0.237605426436

# DBSCAN recall 0.449532057079
# DBSCAN precision 0.654753576815

# Markov Cluster recall 0.407875505471
# Markov Cluster precision 0.369869384098



# >=15 and no past using tweet_id
# Affinity Propagation recall 0.340671369697
# Affinity Propagation precision 0.333549833505

# DBSCAN recall 0.641642903856
# DBSCAN precision 0.804690509565

# Markov Cluster recall 0.520358917544
# Markov Cluster precision 0.505077553767

In [14]:
sorted(true_event.items())

[(datetime.datetime(2012, 6, 1, 0, 0),
  [u'Glass door concussion',
   u'Norway Concert ',
   u'World Tour',
   u"It's All About Money ",
   u'Battery Suspect',
   u'Taxi - Nissan Leafs',
   u'Client protest: yayoflautas',
   u'Achievement - Sales',
   u'Yellow Submarine DVD - Remastered',
   u'Problems in sales',
   u'Achievements']),
 (datetime.datetime(2012, 6, 2, 0, 0),
  [u'Jubilee Concert',
   u'Concierto Gratis en Zocalo',
   u'Electric Cars - To charge homes',
   u'Honda in trouble',
   u'robbery at HSBC',
   u'Unhappy Nissan Drivers',
   u'Stolen cars',
   u'24hr Le Mans']),
 (datetime.datetime(2012, 6, 3, 0, 0),
  [u'Complaint about an auto recall',
   u'Chicago Factory',
   u'Mad Men - Tomorrow Never Knows',
   u'DeltaWing',
   u'Rio Ferdinand Rumoured to sing Beatles',
   u'Nissan Workers Union']),
 (datetime.datetime(2012, 6, 4, 0, 0),
  [u'Achievements',
   u'New Beatles Album Cover',
   u'Stolen cars',
   u'Withheld Loss Figures - Merrill Vote',
   u'James McCartney - Ne

In [15]:
num = 0
for e in true_event.values():
    num += len(e)
print 'num event :',num
print 'event per day :',1.0*num/len(true_event)

num event : 462
event per day : 2.31


In [16]:
# event_size = 10
# num event : 344
# event per day : 1.70297029703

# event_size = 15
# num event : 176
# event per day : 0.871287128713